### 概率图模型

概率图模型（probabilistic graphical model）是一类用图来表达变量相关关系的概率模型，可以大致分为两类：第一类使用有向无环图表示变量间的依赖关系，称为有向图模型或贝叶斯网（Bayesian network）；第二类使用无向图表示变量间的相关关系，称为无向图模型或马尔可夫网（Markov network）。

隐马尔可夫模型（HMM）是结构最简单的动态贝叶斯网，主要用于时序数据建模。

### HMM的基本介绍

![title](HMM_model.PNG)

如图，HMM模型的变量可分为两组。第一组是状态变量$\{y_1,y_2,\cdots,y_n\}=\mathcal{Y}$，其中$y_i$表示第$i$时刻系统状态，状态变量通常不可观测，故也称隐变量。第二组是观测变量$\{x_1,x_2,\cdots,x_n\}$，其中$x_i\in \mathcal{X}$表示第$i$时刻系统状态。

系统状态可在多个状态$\{s_1,s_2,\cdots,s_N\}$之间转换；观测变量可以是离散型也可以是连续型，这里仅考虑离散型，设$\mathcal{X}=\{o_1, o_2,\cdots,o_M\}$。即一共N个隐变量，M个观测变量。

从图中还可以看出变量之间的依赖关系：任意时刻观测变量取值仅依赖其对应的状态变量，同时$y_t$也只依赖于$y_{t-1}$，这就是所谓的“马尔可夫链”。

基于这种依赖，所有变量的联合概率分布为（联合概率目标是机器学习中生成式模型的建模对象）：

$P(x_1,y_1,\cdots,x_n,y_n)=P(y_1)P(y_1|x_1)\prod_{i=1}^{n}P(y_i|y_{i-1})P(x_i|y_i)$

上式右边的$P(y_1)$对应图中根节点，其他每一项对应图中的一条边。

从该联合分布中，可以看出我们想要确定这个联合分布（或者说确定该HMM），还想要以下三组参数：

1. 初始状态概率，模型在初始时刻各状态出现的概率，记为$\pi=(\pi_1,\cdots,\pi_N)$，其中$\pi_i=P(y_1=s_i)$，表示初始状态为$s_i$的概率。

2. 输出观测概率，由当前状态获得各个观测值的概率，记为$B\in R^{N\times M}$，其中$B_{ij}=P(x_t=o_j|y_t=s_i)$，表示由隐变量$s_i$产生观测值$o_j$的概率。

3. 状态转移概率，模型在各个隐藏状态间的转移的概率，记为$A\in R^{N\times N}$，其中$A_{ij}=P(y_{t+1}=s_j|y_{t}=s_i)$，表示从状态$s_i$转移到$s_j$的概率。

所以，一个HMM可以由如下五个参数确定：观测空间$\mathcal{X}$，隐状态空间$\mathcal{Y}$，以及模型参数$\lambda = \{A,B,\pi\}$。

### HMM的三个基本问题

在实际应用中，人们常关注HMM的三个基本问题：

1. 给定$\lambda$，如何计算某观测序列$x=\{x_1,\cdots,x_n\}$的出现概率$P(x|\lambda)$？

2. 给定$\lambda$及观测序列$x=\{x_1,\cdots,x_n\}$，如何找到最匹配的隐状态序列$y=\{y_1,\cdots,y_n\}$？

3. 给定观测序列$x=\{x_1,\cdots,x_n\}$，如何找到使该序列出现概率最大的模型参数$\lambda$？

第一个问题的现实情境为根据之前的观测序列来推测下一刻的观测值，因为我们可以计算各个观测值出现的概率；第二个问题的现实情境为语音识别，即根据语言这种观测序列来推断文字这种隐变量；第三个问题就是如何根据训练样本学得HMM的参数，因为很多问题中，难以靠人工来制定这些参数。

由于HMM的联合概率分布具有条件独立性，这三个问题均可以高效求解。

### 代码实现

这里的问题是：一个人假设由两种隐状态$\mathcal{Y}=\{fever, healthy\}$，有三个观测量$\mathcal{X}=\{normal, dizzy, cold\}$。另外还知道HMM的参数$\lambda=\{A,B,\pi\}$。

![example](HMM_example.PNG)

In [2]:
import numpy as np

In [3]:
"""
    本HMM的五个参数
"""

# 对应状态集合Q
states = ('Healthy', 'Fever')
s = np.array([0, 1])

# 对应观测集合V
observations = ('normal', 'cold', 'dizzy')
o = np.array([0, 1, 2])

# 初始状态概率向量π
start_probability = {'Healthy': 0.6, 'Fever': 0.4}
pi = np.array([0.6, 0.4])

# 状态转移矩阵A
transition_probability = {
    'Healthy': {'Healthy': 0.7, 'Fever': 0.3},
    'Fever': {'Healthy': 0.4, 'Fever': 0.6},
}
A = np.array([[0.7, 0.3],[0.4, 0.6]])

# 观测概率矩阵B
emission_probability = {
    'Healthy': {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
    'Fever': {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
}
B = np.array([[0.5, 0.4, 0.1],[0.1, 0.3, 0.6]])

In [4]:
def draw_from(probs):
        """
            从一个多项式分布中采样
            
            1. 输入probs是骰子各面出现的概率（假设骰子有n面）
            2. np.random.multinomial(1,probs)表示掷一次骰子，
               返回一个长度为n的独热码，被选的那面其值为1，其他值为0
            3. np.where(... == 1)[0][0]返回被选的那一面的下标
        """
        return np.where(np.random.multinomial(1,probs) == 1)[0][0]

In [5]:
def sample(T):
    """
        从定义的HMM中采样出一个长为T的观测序列和与之对应的隐状态序列
    """
    
    sts_sample = []          # 隐状态序列
    obs_sample = []          # 观测序列
    
    s = draw_from(pi)         # 初始隐状态
    sts_sample.append(s)
    o = draw_from(B[s])       # 初始观测状态
    obs_sample.append(o)
    
    for i in range(T-1):
        s = draw_from(A[s])
        sts_sample.append(s)
        o = draw_from(B[s])
        obs_sample.append(o)
        
    return sts_sample, obs_sample

In [6]:
sts_sample, obs_sample = sample(20)

print(sts_sample)
print(obs_sample)

[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1]
[0, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 2, 2, 0, 2, 2, 0, 2, 2, 2]


下面来解决HMM的三个基本问题

#### 1.给定$\lambda$，如何计算某观测序列$x=\{x_1,\cdots,x_n\}$的出现概率$P(x|\lambda)$？

利用前向算法，利用马尔可夫过程的性质——下一刻的状态只与上一刻有关。每一刻的隐状态都有N种可能，这里用$\alpha_t(i)$表示第t时刻隐状态为i时且出现前t个观测序列的联合概率，即：

$\alpha_t(i)=P(x_1,\cdots,x_t,y_t=i|\lambda)$

所以最终要求的$P(x|\lambda)$就是$t=n$时$\alpha_t(i)$对所有可能i的求和：

$P(x|\lambda)=\sum_{i=1}^N\alpha_n(i)$

$\alpha_t(i)$只与上个状态的$\alpha_{t-1}(j)$有关：

$\alpha_t(i)=(\sum_{j=1}^N\alpha_{t-1}(j)\cdot A_{ji})B_{i,x_t}$

$\alpha_1(i)=\pi_i \cdot B_{i,x_1}$

右侧括号内表示从上一刻所有可能隐状态到当前状态i的概率，即当前隐状态i出现概率，$B_{i,x_i}$就是由当前状态i产生观测值$x_i$的概率。由于$\alpha_t$的推导是从前向后依次进行，故称前向算法。

In [7]:
def forward(obs_seq):
    """
        前向算法
        
        输入观测到的序列
        输出该序列出现的概率
    """
    N = A.shape[0]    # 隐状态个数
    T = len(obs_seq)  # 观测到的序列长度
    
    # 前向概率矩阵，共T行N列
    # 第i行第j列的元素表示：第i个隐状态为j时前i个观测值出现的概率
    # 故第i行所有元素的和即为前i个观测值出现的概率
    F = np.zeros((T,N)) 
    
    # 初始前向概率（第一个观测值由各个隐状态生成的概率）
    F[0, :] = pi * B[:, obs_seq[0]]
    
    for t in range(1, T):
        for cur_s in range(N):
            
            cur_s_prob = 0             # 从上一刻隐状态转移到当前隐状态cur_s的概率
            for prev_s in range(N):    # 上一刻状态为prev_s
                cur_s_prob += F[t-1, prev_s] * A[prev_s, cur_s]
                
            # 当前隐状态为cur_s的概率
            F[t, cur_s] = cur_s_prob * B[cur_s, obs_seq[t]]
            
    obs_seq_prob = sum(F[-1, :])
    
    return F, obs_seq_prob

In [8]:
F, obs_seq_prob = forward(obs_sample)
print(obs_sample)
print(F)
print(obs_seq_prob)

[0, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 2, 2, 0, 2, 2, 0, 2, 2, 2]
[[3.00000000e-01 4.00000000e-02]
 [9.04000000e-02 3.42000000e-02]
 [3.07840000e-02 1.42920000e-02]
 [1.36328000e-02 1.78104000e-03]
 [5.12768800e-03 5.15846400e-04]
 [1.51828806e-03 5.54344272e-04]
 [5.13815741e-04 2.36427895e-04]
 [1.81696871e-04 8.88004378e-05]
 [6.50831939e-05 3.23367972e-05]
 [5.84929546e-06 2.33562219e-05]
 [5.37479823e-06 4.73055653e-06]
 [5.65458137e-07 2.67046403e-06]
 [1.46400631e-07 1.06314952e-06]
 [2.63870124e-07 6.81809899e-08]
 [2.11981483e-08 7.20417787e-08]
 [4.36554153e-09 2.97507070e-08]
 [7.47808094e-09 1.91600867e-09]
 [6.00106012e-10 2.03581769e-09]
 [1.23440128e-10 8.40913450e-10]
 [4.22773470e-11 3.24948065e-10]]
3.6722541213200044e-10


#### 2.给定$\lambda$及观测序列$x=\{x_1,\cdots,x_n\}$，如何找最匹配的隐状态序列$y=\{y_1,\cdots,y_n\}$？

这里使用Viterbi算法。同样利用马尔可夫链的性质，我们可以建立一个仅与上个状态有关的递归结果。假设时刻t时隐变量的序列为$\{y_1,\cdots,y_t\}$，这里用到的递推式如下：

$\delta_{t}(i)=\max_{y_1,\cdots,y_{t-1}}P(y_t=i,y_1,\cdots,y_{t-1},x_1,\cdots,x_t|\lambda)$

即：t时刻隐变量为i且观测到前t个观测值的最大联合概率。递推关系如下：

$\delta_t(i)=\max_{1\leq j\leq N}(\delta_{t-1}(j)A_{ji})B_{i,o_t}$

$\delta_1(i)=\pi_i B_{i,o_1}$

要想获得最佳隐状态序列，只与记录t时刻概率最大的隐变量的下标即可。

In [16]:
def Viterbi(obs_seq):
    """
        Viterbi算法
        
        输入观测到的序列
        输出最可能的隐变量序列
    """
    N = A.shape[0]    # 隐状态个数
    T = len(obs_seq)  # 观测到的序列长度
    
    # t时刻隐变量为i的最大概率矩阵，共T行N列
    delta = np.zeros((T,N))
    
    # 最可能的隐变量序列
    path = np.zeros(T)
    
    delta[0, :] = pi * B[:, obs_seq[0]]
    path[0] = np.argmax(delta[0, :])
    
    for t in range(1, T):
        for i in range(N):
            
            # 计算t时刻隐变量为i的最大概率
            temp = np.zeros(N)              
            for j in range(N):        # t时刻状态为i，t-1时刻状态为j
                temp[j] = delta[t-1, j] * A[j, i]
                
            delta[t, i] = np.max(temp) * B[i, obs_seq[t]]
            
        path[t] = np.argmax(delta[t, :])
    
    return path

In [17]:
path = Viterbi(obs_sample)
print(path)

# 下面是ground truth
print(sts_sample)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1.]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1]


##### 问题1和2的递推式非常相似，都是时刻t时与隐状态i和已观测的值的联合概率。这个联合概率的大小来自两处：一处是前一刻的状态j，j越可能转移到i，i出现的概率越大；另一处是当前的观测值，从i到观测值o的概率越大，i出现的概率也就越大。二者缺一不可。这个过程用数学表示就是：

##### $P(X,Y|\lambda)=P(X|Y,\lambda)P(Y|\lambda)$

#### 3.给定观测序列$x=\{x_1,\cdots,x_n\}$，如何找到使该序列出现概率最大的模型参数$\lambda$？